# Big-M Method
Used when we can't use simplex method.

- The Big-M method is a variant of the simplex algorithm used to handle linear programming problems with artificial variables.

- Typically used when some constraints are equalities (=) or greater-than-or-equal-to (≥). In this case, artificial variables are introduced to help start the simplex method, and these artificial variables are penalized in the objective function using a large number, 
𝑀 to drive them out of the solution.

In [37]:
import numpy as np
from tabulate import tabulate


def print_tableau(tableau, basis, n, m, z, inequalities):
    """
    Function to print the current tableau in tabular form.
    """
    num_art_vars = np.count_nonzero(np.array(inequalities) != 'leq')
    headers = ["Basic", "P"] + [f"X{i+1}" for i in range(n)] + \
              [f"S{i+1}" for i in range(z)] + \
              [f"A{i+1}" for i in range(num_art_vars)] + ["Solution"]
    rows = []

    for i in range(m + 1):
        if i < m:
            basic_var_index = basis[i]
            if basic_var_index < n:
                basic_var = f"X{basic_var_index+1}"
            elif basic_var_index < n + z:
                basic_var = f"S{basic_var_index - n + 1}"
            else:
                basic_var = f"A{basic_var_index - n - (m - num_art_vars) + 1}"
            
            row = [f"Row {i+1}", basic_var] + [f"{tableau[i, j]:.2f}" for j in range(n + m + num_art_vars)]
        else:
            row = ["Objective", "P"] + [f"{tableau[i, j]:.2f}" for j in range(n + m + num_art_vars)]

        rows.append(row)

    print(tabulate(rows, headers=headers, tablefmt="pipe"))
    print()


def big_m_simplex(c, A, b, inequalities, M=1e6):
    """
    Parameters:
    c (numpy array): Coefficients of the objective function.
    A (numpy array): Coefficients of the constraint inequalities.
    b (numpy array): Right-hand side values of the constraints.
    inequalities (list): List of inequality directions for constraints.
                         Use 'leq' for ≤, 'geq' for ≥, and 'eq' for =.
    M (float): A large value for the Big-M penalty.
    
    Returns:
    optimal_solution (numpy array): Optimal solution to the decision variables.
    optimal_value (float): Optimal value of the objective function.
    """
    
    m, n = A.shape
    
    # Add slack variables for '≤' constraints, and artificial variables for '≥' and '=' constraints
    tableau = np.zeros((m + 1, n + m + np.count_nonzero(np.array(inequalities) != 'leq')))
    tableau[:m, :n] = A
    tableau[:m, -1] = b
    slack_index = 0 
    art_var_indices = []
    row = []
    basic_var = []  # Store column indexes of basic variables

    z = np.count_nonzero(np.array(inequalities) != 'eq')

    for i, inequality in enumerate(inequalities):
        if inequality == 'leq':  # ≤: Add slack variables
            tableau[i, n + slack_index] = 1
            basic_var.append(n + slack_index)
            slack_index += 1
        elif inequality == 'geq':  # ≥: Add slack variables (negative) and artificial variables
            basic_var.append(n + z + len(art_var_indices))
            tableau[i, n + slack_index] = -1
            tableau[i, n + z + len(art_var_indices)] = 1
            art_var_indices.append(n + z + len(art_var_indices))
            row.append(slack_index)
            slack_index += 1
        elif inequality == 'eq':  # =: Add artificial variables
            tableau[i, n + z + len(art_var_indices)] = 1
            basic_var.append(n + z + len(art_var_indices))
            art_var_indices.append(n + z + len(art_var_indices))
            row.append(slack_index)
            slack_index += 1

    # Add the objective function (maximize: -c, minimize: c)
    tableau[-1, :n] = -c
    tableau[-1, art_var_indices] = M 

    for i in range(len(art_var_indices)):
        tableau[-1, :] -= M * tableau[row[i], :]
    
    basis = basic_var
    
    print("Initial Tableau:")
    print_tableau(tableau, basis, n, m, z, inequalities)

    # Start the simplex method iterations
    iteration = 0
    while np.min(tableau[-1, :-1]) < 0:
        iteration += 1
        print(f"Iteration {iteration}:")

        pivot_col = np.argmin(tableau[-1, :-1])
        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        positive_ratios = ratios > 0
        if not np.any(positive_ratios):
            raise Exception("Problem is unbounded")

        positive_indices = np.where(positive_ratios)[0]
        min_positive_index = positive_indices[np.argmin(ratios[positive_ratios])]
        pivot_row = min_positive_index
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element

        for i in range(m + 1):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

        basis[pivot_row] = pivot_col

        print_tableau(tableau, basis, n, m, z, inequalities)

    optimal_solution = np.zeros(n)
    for i in range(m):
        if basis[i] < n:
            optimal_solution[basis[i]] = tableau[i, -1]

    return optimal_solution, tableau[-1, -1]


## Q2 - Big M-Max

In [38]:
c = np.array([6, -7, -4])  # Coefficients of the objective function
A = np.array([[2, 5, -1], [-1, 1, 2], [3, 2, 2]])  # Coefficients of the constraints
b = np.array([18, 14, 26])# Right-hand side of the constraints
inequalities = ['leq', 'geq', 'eq']

solution, optimal_value = big_m_simplex(c, A, b, inequalities)

print("Optimal Solution:", solution)
print("Optimal Value:", optimal_value)


Initial Tableau:
| Basic     | P   |           X1 |           X2 |     X3 |   S1 |     S2 |   A1 |   A2 |   Solution |
|:----------|:----|-------------:|-------------:|-------:|-----:|-------:|-----:|-----:|-----------:|
| Row 1     | S1  |  2           |  5           | -1     |    1 |  0     |    0 |    0 |     18     |
| Row 2     | A2  | -1           |  1           |  2     |    0 | -1     |    1 |    0 |     14     |
| Row 3     | A3  |  3           |  2           |  2     |    0 |  0     |    0 |    1 |     26     |
| Objective | P   | -2.00001e+06 | -2.99999e+06 | -4e+06 |    0 |  1e+06 |    0 |    0 |     -4e+07 |

Iteration 1:
| Basic     | P   |     X1 |        X2 |   X3 |   S1 |        S2 |     A1 |   A2 |   Solution |
|:----------|:----|-------:|----------:|-----:|-----:|----------:|-------:|-----:|-----------:|
| Row 1     | S1  |  1.5   |       5.5 |    0 |    1 |      -0.5 |  0.5   |    0 |   25       |
| Row 2     | X3  | -0.5   |       0.5 |    1 |    0 |      -0.5 |  0